In [ ]:
#Import
import pandas as pd
import re
import matplotlib.pyplot as plt

#set totals to 0
total_calories = 0
total_fat = 0
total_protein = 0
total_carbs = 0
total_sugar = 0
total_fiber = 0

#Overwrite File
df = pd.read_csv("data.csv")

file_reset = open('foodgroup.csv','w')
file_reset.write("Food Group\n")
file_reset.close()


while True:
    continue_search = True
    
    #User inputs food item
    while True: 
        try:
            search = input("Type food item: ")   
    
            #Searches database and finds most common/relevant food group
            search_df = df.loc[df["Name"].str.contains(search, flags=re.I, regex=True)]
      
            search_df["count"] = 1
            search_df.groupby(["Food Group"]).count()["count"]
            sort_df = search_df[['Food Group','count']].groupby(['Food Group'])['count'] \
                                     .count() \
                                     .reset_index(name='count') \
                                     .sort_values(['count'], ascending=False) \
                                     .head(1)
            sort_df = sort_df.reset_index(drop=True)
            sort_df = (sort_df.iloc[0,0])
            
        except: 
            print ("No results found.")
            continue
        else:
            break 
            
    compact_df = df.loc[(df["Name"].str.contains(search, flags=re.I, regex=True)) & (df["Food Group"] == sort_df)]
    cal = compact_df.iloc[:,3].mean()
    fat = compact_df.iloc[:,4].mean()
    protein = compact_df.iloc[:,5].mean()
    carbs = compact_df.iloc[:,6].mean()
    sugar = compact_df.iloc[:,7].mean()
    fiber = compact_df.iloc[:,8].mean()      
    
    while True:
        try:
            serving_size = int(input("Please enter your serving size (grams) \n"))
        except:
            print ("Invalid input \n")
            continue
        else:
            break
    
    total_calories = total_calories + (cal * (serving_size/100))
    total_fat = total_fat + (fat * (serving_size/100))
    total_protein = total_protein + (protein * (serving_size/100))
    total_carbs = total_carbs + (carbs * (serving_size/100))
    total_sugar = total_sugar + (sugar * (serving_size/100))
    total_fiber = total_fiber + (fiber * (serving_size/100))
       
    #Appends food group to sheet
    file = open('foodgroup.csv','a')
    sort_df = sort_df.strip()
    file.write(sort_df)
    file.write("\n")
    file.close()
    
    #Ask user if they want to continue
    while continue_search is True:
    
        continue_search_box = input("Would you like to add more food items? (Y/N) \n")
        if continue_search_box.upper() == "Y" or continue_search_box.upper() == "N":
            continue_search = False
        else:
            print("Incorrect input \n")
    
    if continue_search_box.upper() == "N":
        break

        
#Tallying 
fg_df = pd.read_csv("foodgroup.csv")
fg_df["count"] = 1
fg_df.groupby(["Food Group"]).count()["count"]
food_df = fg_df[['Food Group','count']].groupby(['Food Group'])['count'] \
                                     .count() \
                                     .reset_index(name='count') \
                                     .sort_values(['count'], ascending=False)


#Pie Chart
food_group = food_df["Food Group"]
count = food_df["count"]
plt.pie(count, labels=food_group, shadow=False)
plt.title("Is Your Meal Balanced?")
plt.show()



#Bar chart for daily intake
#Average NRV from Google search
nrv_calories = 2500 #kcal
nrv_fat = 90 #g
nrv_protein = 48 #g
nrv_carbs = 325 #g
nrv_sugar = 37.5 #g
nrv_fiber = 35 #g

#Percentages
p_cal = (total_calories / nrv_calories) * 100
p_fat = (total_fat / nrv_fat) * 100
p_protein = (total_protein / nrv_protein) * 100
p_carbs = (total_carbs / nrv_carbs) * 100
p_sugar = (total_sugar / nrv_sugar) * 100
p_fiber = (total_fiber / nrv_fiber) * 100

#Bar Chart
fig = plt.figure()
parameters = ["Calories", "Fat", "Protein", "Carbohydrates", "Sugar", "Fiber"]
values = [p_cal, p_fat, p_protein, p_carbs, p_sugar, p_fiber]
ax = fig.add_axes([0,0,1,1])
ax.bar(parameters,values)
ax.set_ylabel('Recommended Daily Intake (%)')
ax.set_title('Representation of your Daily Nutrtional Intake')
plt.show()


